In [1]:
%load_ext autoreload
%autoreload 2

import spglib as spg
import ase
from ase.visualize import view

import ase.spacegroup as aspg
import numpy as np
import pandas as pd
from collections import Iterable
import cPickle as pck
import spglib as spg
import sys, os
from utils import rotation_matrix,isCellSkewed,unskewCell,get_symprec,get_relative_angle
from qe_input import makeCard,makeNamelist,SG2ibrav,SG2wyckoff,makeQEInput_sg,makeQEInput_ibrav0,makeQEInput
from custom_frame import frame2qe_format
from qe_run import run_qe_local,make_submit_script
from raw_info import SG2BravaisLattice
from SSSP_acc_PBE_info import PP_names,rhocutoffs,wfccutoffs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# local execution

In [ ]:
os.environ['TMPDIR'] = "/tmp/"

In [3]:
def make_dir(file_path):
    directory = os.path.abspath(file_path)
    suffix = 0
    while os.path.exists(directory+'-{}'.format(suffix)):
        suffix += 1
    path = directory +'-{}/'.format(suffix)
    os.makedirs(path)
    return path

def run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=2,path2pw='/home/musil/source/qe-6.1/bin/'):

    path = make_dir(dirName)
    inputName = os.path.abspath(path+'/qe.in')
    outputName = os.path.abspath(path+'/qe.out')
    errName = os.path.abspath(path+'/qe.err')
    with open(inputName,'w') as f:
        f.write(input_str)
    param = '-in '+ inputName
    if verbose:
        print param
    # Set up the echo command and direct the output to a pipe
    exitState = sp.call('{}mpirun -np {:.0f} {}pw.x {}'.format(path2mpi,np,path2pw,param)
                            , stdout=open(outputName, 'w'),
                            stderr=open(errName, 'w'), shell=True)
    error = 'No error'
    if exitState:
        print dirName
        with open(outputName,'r') as f:
            error = f.read()

    return exitState,error

# hpc execution

In [5]:
submit_cmd = {'normal':'sbatch ',
              'debug':'sbatch --partition=debug '}

In [7]:
dirName = './'

In [9]:
path = make_dir(dirName)
inputName = os.path.abspath(path+'/qe.in')
submit_scriptName = os.path.abspath(path+'/qe.sh')
outputName = os.path.abspath(path+'/qe.out')
jobName = os.path.abspath(path+'/job.id')
errName = os.path.abspath(path+'/job.err')

submit_script = make_submit_script(hpc='deneb', input_fn=inputName, output_fn=outputName,
                                   workdir=path, node=1, tasks=1,
                                   cpus_per_tasks=1, mem=63000, time='00:10:00', debug=debug)


print submit_script
# with open(inputName,'w') as f:
#     f.write(input)
# with open(submit_scriptName,'w') as f:
#     f.write(submit_script)

# Set up the echo command and direct the output to a pipe
# exitState = sp.call('sbatch {}'.format(submit_scriptName), 
#                     stdout=open(jobName, 'a'),stderr=open(errName, 'w'),shell=True)



#!/bin/bash 
#SBATCH --workdir /local/git/run_qe/make_input-1/ 
#SBATCH --nodes 1 
#SBATCH --ntasks 1 
#SBATCH --cpus-per-task 1 
#SBATCH --mem 63000 
#SBATCH --time 00:10:00  
#SBATCH --constrain=E5v2 

module purge 
module load intel/17.0.2  intel-mpi/2017.2.174 intel-mkl/2017.2.174 

srun /home/musil/source/qe-6.1/bin/pw.x -in /local/git/run_qe/make_input-1/qe.in > /local/git/run_qe/make_input-1/qe.out 



In [ ]:
def run_qe_hpc(input_str,dirName,verbose=False,hpc='deneb', node=1, tasks=1,
                cpus_per_tasks=1, mem=63000, time='00:10:00', debug=False):
    path = make_dir(dirName)
    inputName = os.path.abspath(path+'/qe.in')
    submit_scriptName = os.path.abspath(path+'/qe.sh')
    outputName = os.path.abspath(path+'/qe.out')
    jobName = os.path.abspath(path+'/job.id')
    errName = os.path.abspath(path+'/job.err')

    submit_script = make_submit_script(hpc=hpc, input_fn=inputName, output_fn=outputName,
                                       workdir=path, node=node, tasks=tasks,
                                       cpus_per_tasks=cpus_per_tasks, mem=mem, time=time, debug=debug)

    with open(inputName,'w') as f:
        f.write(input)
    with open(submit_scriptName,'w') as f:
        f.write(submit_script)

    #  Set up the echo command and direct the output to a pipe
    exitState = sp.call('sbatch {}'.format(submit_scriptName), 
                        stdout=open(jobName, 'a'),stderr=open(errName, 'w'),shell=True)
    return exitState